# Setup environment

In [1]:
import datetime
from dotenv import load_dotenv, find_dotenv
import os
from typing import List, Dict, Any
import pprint

In [2]:
# https://pypi.org/project/openai/
import openai
from openai.openai_object import OpenAIObject

In [3]:
_ = load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY', 'MISSING_OPENAI_API_KEY')

# Prompt LLM

## Create helpers

In [4]:
def get_completion(prompt: str, model: str, temperature: float, n_completions: int) -> str:
    messages: List[Dict[str, Any]] = [
        {
            "role": "user",
            "content": prompt
        }
    ]
    try:
        response: OpenAIObject = openai.ChatCompletion.create(
            messages=messages,
            model=model,
            temperature=temperature,
            n=n_completions
        ) 
    except Exception as e:
        return f"Error: {e}"

    if not response.choices:
        return "Error: No completions returned"

    return response.choices[0].message.content

## Set prompting params

In [5]:
# https://platform.openai.com/docs/models/model-endpoint-compatibility
completion_model = 'gpt-3.5-turbo'

In [6]:
# https://platform.openai.com/docs/api-reference/completions/create#completions/create-temperature
temperature = 0.01

In [7]:
# https://platform.openai.com/docs/api-reference/completions/create#completions/create-n
n_completions = 1

In [8]:
prompting_params: Dict[str, Any] = {
    "model": completion_model,
    "temperature": temperature,
    "n_completions": n_completions
}

## Start prompting

### Prompting Principle 1: Write clear and specific instructions

Tactics in the principle:
1. Use delimiters to clearly indicate distinct parts of the input
2. Ask for a structured output
3. Ask the model to check whether conditions are satisfied
4. "Few-shot" prompting

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

In [9]:
text = f"""
Prom.ua — український маркетплейс, проєкт IT-компанії EVO.
На його платформі підприємці[1] самостійно створюють інтернет-магазини і / або розміщують свої товари в загальному каталозі.
Для покупців на Промі зібрано більше 100 мільйонів товарів.
В 2018 році українці витратили на маркетплейсі 8,6 млрд грн — і ще 10,5 млрд грн на сайтах компаній, створених на базі Prom.ua[3].
Щодня торговельний майданчик відвідує близько 2 млн користувачів.
У ТОП-10 популярних категорій входять одяг, взуття, аксесуари, товари для дому та саду, краси та здоров'я, а також техніка й електроніка[4].
Щомісяця на маркетплейсі покупці оформлюють у середньому 1,5 млн замовлень на суму близько 1,3 млрд грн[5].
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""
response = get_completion(prompt, **prompting_params)
pprint.pprint(response)

('Prom.ua is a Ukrainian marketplace where entrepreneurs can create their own '
 'online stores or list their products in a shared catalog, with over 100 '
 'million products available for purchase and approximately 2 million daily '
 'visitors, with popular categories including clothing, footwear, accessories, '
 'home and garden products, beauty and health, and electronics, resulting in '
 'an average of 1.5 million orders per month worth around 1.3 billion UAH.')


#### Tactic 2: Ask for a structured output
- use JSON, HTML

In [10]:
prompt = f"""
Generate a list of top 5 Ukrainian e-commerce businesses working in 2018.

Provide them in JSON format with the following keys: 
business_id, name, ceo, users.
"""
response = get_completion(prompt, **prompting_params)
pprint.pprint(response)

('[\n'
 '  {\n'
 '    "business_id": 1,\n'
 '    "name": "Rozetka",\n'
 '    "ceo": "Vladimir Kostevich",\n'
 '    "users": 10,000,000\n'
 '  },\n'
 '  {\n'
 '    "business_id": 2,\n'
 '    "name": "Epicentr K",\n'
 '    "ceo": "Vladimir Kostevich",\n'
 '    "users": 5,000,000\n'
 '  },\n'
 '  {\n'
 '    "business_id": 3,\n'
 '    "name": "Allo",\n'
 '    "ceo": "Vladimir Kostevich",\n'
 '    "users": 3,000,000\n'
 '  },\n'
 '  {\n'
 '    "business_id": 4,\n'
 '    "name": "Foxtrot",\n'
 '    "ceo": "Vladimir Kostevich",\n'
 '    "users": 2,000,000\n'
 '  },\n'
 '  {\n'
 '    "business_id": 5,\n'
 '    "name": "Comfy",\n'
 '    "ceo": "Vladimir Kostevich",\n'
 '    "users": 1,500,000\n'
 '  }\n'
 ']')


#### Tactic 3: Ask the model to check whether conditions are satisfied

In [11]:
text = f"""
Hello! My name is Max and I am writing a text for Tactic 3 section.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 

If it contains a word ROZETKA, re-write the text, replacing each vowel with ROZETKA.
If the text does not contain a word ROZETKA, then simply write \"No ROZETKA for now\"

\"\"\"{text}\"\"\"
"""
response = get_completion(prompt, **prompting_params)
pprint.pprint(response)

'No ROZETKA for now'


In [21]:
text = f"""
Hello! My name is Max and I am writing a text for Tactic 3 section.ROZETKA
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 

If it contains a word ROZETKA, re-write the text, replacing each vowel with ROZETKA.
If the text does not contain a word ROZETKA, then simply write \"No ROZETKA for now\"

\"\"\"{text}\"\"\"
"""
response = get_completion(prompt, **prompting_params)
pprint.pprint(response)

('HOROZETKOlROZETKA! My nROZETKAme ROZETKAis MROZETKAx ROZETKAand ROZETKAI '
 'ROZETKAam writing ROZETKAa text for TROZETKActic 3 sectiROZETKAon.')


#### Tactic 4: "Few-shot" prompting

In [22]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about e-commerce business.

<ceo>: E-commerce business is all about selling products or services \
online through a website or mobile app. To be successful, e-commerce \
businesses need to have a user-friendly website, competitive pricing, \
good marketing strategies, and excellent customer service. E-commerce \
has become a massive industry, with global sales surpassing $4.2 \
trillion in 2020. With clever and innovative tactics, some e-commerce \
businesses have found ways to stand out from the competition. Ultimately, \
e-commerce business is all about providing convenience and value to \
customers in the digital age.

<child>: Teach me about Ukrainian e-commerce.
"""
response = get_completion(prompt, **prompting_params)
pprint.pprint(response)

('<ceo>: Ukrainian e-commerce is a rapidly growing industry that has seen '
 'significant growth in recent years. The country has a population of over 42 '
 'million people, with a high percentage of internet users, making it an '
 'attractive market for e-commerce businesses. Ukrainian e-commerce businesses '
 'offer a wide range of products and services, including clothing, '
 'electronics, and food delivery. To be successful in the Ukrainian e-commerce '
 'market, businesses need to have a strong online presence, offer competitive '
 'pricing, and provide excellent customer service. The Ukrainian government '
 'has also implemented policies to support the growth of e-commerce, such as '
 'reducing taxes and simplifying regulations. Overall, Ukrainian e-commerce '
 'presents a great opportunity for businesses looking to expand their reach '
 'and tap into a growing market.')


### Prompting Principle 2: Give the model time to “think”

Tactics in the principle:
1. Specify the steps required to complete a task
2. Instruct the model to work out its own solution before rushing to a conclusion

#### Tactic 1: Specify the steps required to complete a task

In [24]:
text = f"""
Усі 15 років піклуємося про твій час, тому на Промчику ти можеш знайти все необхідне, а ми постійно вдосконалюємо процеси замовлення.

Сьогодні приймаємо вітання, але також даруємо тобі можливість виграти смарт-годинник Gelius Pro GP-SW004🥳

Щоб взяти участь у розіграші, потрібно:
✅ Підписатися на @prom.pompom
✅ Тегнути у коментарях друга

Чим більше тегів друзів, тим більше шансів отримати приз😊

Результат оголосимо в сторіз вже 5.05 за допомогою рандомайзера.

Офіційні умови розіграшу закріпили за посиланням у шапці профілю.
Щасти 🤞🏻
"""

In [28]:
# example 1
prompt = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into English.
3 - List each online account name in the English summary.
4 - Output a json object that contains the following \
keys: english_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt, **prompting_params)
# pprint.pprint(response)
print(response)

1 - Ukrainian online store Promchik is celebrating its 15th anniversary by offering customers the chance to win a Gelius Pro GP-SW004 smartwatch by subscribing to their @prom.pompom account and tagging a friend in the comments section of their post.
2 - Promchik, a Ukrainian online store, is celebrating its 15th anniversary by offering customers the chance to win a Gelius Pro GP-SW004 smartwatch by subscribing to their @prom.pompom account and tagging a friend in the comments section of their post.
3 - @prom.pompom
4 - {
     "english_summary": "Promchik, a Ukrainian online store, is celebrating its 15th anniversary by offering customers the chance to win a Gelius Pro GP-SW004 smartwatch by subscribing to their @prom.pompom account and tagging a friend in the comments section of their post.",
     "num_names": 1
   }


In [29]:
# example 2
prompt = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into English.
3 - List each name in the English summary.
4 - Output a json object that contains the 
  following keys: english_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in Italian summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt, **prompting_params)
# pprint.pprint(response)
print(response)

Summary: Promchik is celebrating its 15th anniversary and giving away a Gelius Pro GP-SW004 smartwatch to participants who follow and tag a friend on their Instagram page, with the winner being announced on May 5th through a randomizer. 
Translation: Promchik is celebrating its 15th anniversary and giving away a Gelius Pro GP-SW004 smartwatch to participants who follow and tag a friend on their Instagram page, with the winner being announced on May 5th through a randomizer.
Names: Promchik
Output JSON: {"english_summary": "Promchik is celebrating its 15th anniversary and giving away a Gelius Pro GP-SW004 smartwatch to participants who follow and tag a friend on their Instagram page, with the winner being announced on May 5th through a randomizer.", "num_names": 1}


#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

#### Note that the student's solution is actually not correct.
#### We can fix this by instructing the model to work out its own solution first.

In [30]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt, **prompting_params)
pprint.pprint(response)

"The student's solution is correct."


In [31]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt, **prompting_params)
pprint.pprint(response)

"The student's solution is correct."


### Model limitations: Hallucinations

In [32]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Quick Payment service by prom.ua
"""
response = get_completion(prompt, **prompting_params)
pprint.pprint(response)

("As an AI language model, I don't have access to the latest information about "
 'AeroGlide UltraSlim Smart Quick Payment service by prom.ua. However, based '
 'on my research, AeroGlide UltraSlim Smart Quick Payment service is a payment '
 'solution offered by prom.ua, an e-commerce platform in Ukraine. This service '
 'allows customers to make quick and secure payments using their mobile '
 'devices. The UltraSlim Smart Quick Payment service is designed to be '
 'user-friendly and convenient, allowing customers to complete transactions '
 'with just a few taps on their smartphones. The service is also said to be '
 'highly secure, with advanced encryption and fraud prevention measures in '
 'place to protect customer data and prevent unauthorized access. Overall, '
 'AeroGlide UltraSlim Smart Quick Payment service is a promising payment '
 'solution that could help streamline the online shopping experience for '
 'customers in Ukraine.')


In [33]:
print(datetime.datetime.now())

2023-05-01 16:08:12.193340
